In [1]:
import pandas as pd
from astroquery.simbad import Simbad
from jwst import *

%load_ext autoreload
%autoreload 2
Simbad.add_votable_fields('flux(K)','sp')

max_bex_age = 1000 #This isn't the actually maximum age, but the maximum age above which there isn't much data. 

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Directories to change depending on what computer you're on

In [2]:
figure_base_directory = "./FarOut_Figures" #Change this to reflect the project we're working on
bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'

In [3]:
computer = 'connor'
connor = True
maxmb = False

if computer == 'connor':
    import os
    base_model_dir="/Users/connorvancil/Desktop/AstroResearch/Data/models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'
    os.environ['PYSYN_CDBS']='/Users/connorvancil/Desktop/AstroResearch/Data/trds/'
elif computer == 'max_laptop': 
    base_model_dir="/Users/maxwellmb/Data/models/ATMO_2020_models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'
elif computer == 'max_desktop':
    base_model_dir = "TBD"
    bex_file = None

Read in the Data Table

In [43]:
df = pd.read_csv('../large_orbit_targetlist.csv')

data_arr = df[['hostname','sy_dist','st_age','pl_bmassj','pl_orbsmax']].to_numpy()

target_dict={}
for row in data_arr:
    row[2]*=1000 #convert sy age from Gyr to Myr
    target_dict[row[0]]=row[1:]


Query all targets

In [70]:
targets=df['hostname'].to_numpy()

queried_params=[];spts=[]
for target in targets:
    result_table = Simbad.query_object(target)
    spts.append(result_table['SP_TYPE'][0])
    queried_params.append([target,result_table['FLUX_K'][0]])

Force Spectral Types to those in ck04models

In [90]:
spts[36] = 'F5V';spts[50] = 'K7V';spts[61] = 'M0V';spts[62] = 'M0V';spts[63] = 'M0V';spts[64] = 'A4V';spts[65] = 'K2III';spts[70] = 'K7';spts[78] = 'K7';spts[80] = 'F6V';spts[99] = 'M7';spts[114] = 'F0';
spts[122] = 'K1';spts[126] = 'K0';spts[129]='G0V';spts[95] = 'M3'; spts[102] = 'A0V'
from models import *
for i,spt in enumerate(spts):
    if spt not in lookuptable: 
        spts[i] = match_spt(spt)
[params.append(spts[i]) for i, params in  enumerate(queried_params)]

#adding another dictionary to hold these queried paramters in a more useful format
simbad_vals={}
for star in queried_params:
    simbad_vals[star[0]]=[star[1],star[2]]

Set up some configuration info

In [91]:
configs = {'f250m':  {'instrument':'nircam','instrument_mask':'mask335r'},
            'f300m': {'instrument':'nircam','instrument_mask':'mask335r'}, 
            'f356w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f410m': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f444w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f1140c':{'instrument':'miri',  'instrument_mask':None},
            'f1550c':{'instrument':'miri',  'instrument_mask':None},
            }
filter_list = list(configs.keys())
filter_list = ['f444w'] #For now only doing this filter because the bex filters only have this filter.

In [92]:
full_detection_list = []

for i, target in enumerate(targets):
# for i, target in enumerate([targets[3]]):
    
    age = target_dict[target][1]
    distance = target_dict[target][0]
    
    comp_AU = target_dict[target][3] #Companion separation in AU
    comp_sep = comp_AU/distance #convert AU to arcsec
    comp_mass = target_dict[target][2] #Companions mass in M_Jup

    kmag,spt = simbad_vals[target]
    
    mass_limits_list = []
    sep_list = []
    target_detection_array = []
    almost_detected_array = []

    print("Target {}: {}, {} Myr, {:.0f} pc, kmag={:.1f}".format(i,target,age,distance,kmag))

    if comp_sep < 0.4:
        print("\tCompanion separation for {} is < 0.4 arcseconds, skipping".format(target))
        continue

    for filter_name in filter_list:
        
        instrument = configs[filter_name]['instrument']
        instrument_mask = configs[filter_name]['instrument_mask']

        jwst_mag = get_jwst_mag(spt,kmag,instrument,filter_name,filter_dir="./",jwst_mask=instrument_mask,
                    norm_filter='bessel_k').value

        contrast_curves=read_contrast_curves()
        sep,companion_mags=companion_detection_limit(jwst_mag,filter_name,contrast_curves,plot=False);

        good_seps = sep[np.where(np.isfinite(companion_mags))]

        if comp_sep < np.min(good_seps):
            print("\tCompanion of {:.2f} for {} is closer than the IWA in {}, skipping".format(comp_sep,target,filter_name))
            continue

        model_dir = base_model_dir+model_instrument_directories.get(instrument.lower())
        
        if instrument.lower () == "nircam":
            model_dir+=model_nircam_mask_directories.get(instrument_mask.lower())

        if age < max_bex_age:
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,bex_model_filename=bex_file,
                            plot=False)
        else: 
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,
                            plot=False)

        if len(mass_limits) < len(sep):
            sep = sep[:len(mass_limits)]
        detected = detect_companion(sep,mass_limits,comp_sep,comp_mass)
        target_detection_array.append(detected)

        #What if we got 2x deeper?
        almost_detected = detect_companion(sep,mass_limits/2.,comp_sep,comp_mass)
        almost_detected_array.append(almost_detected)

        # print("Was the companion detected? {}".format())
        mass_limits_list.append(mass_limits)
        sep_list.append(sep)

    if np.any(target_detection_array):
        print("\tCOMPANION DETECTED!")
        full_detection_list.append(True)
        figure_directory = figure_base_directory+"/Detected/"
    elif np.any(almost_detected_array): 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Almost_Detected/"
    else: 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Non-Detected/"
    
    #If the companion is too late at all filters then skip
    if len(sep_list) < 1:
        continue

    fig,axis = plt.subplots(1,1,figsize=(12,5))
    for i in range(len(mass_limits_list)):
        plt.semilogy(sep_list[i],mass_limits_list[i],label=filter_list[i])
    plt.title("{}, {} Myr, {} pc, kmag={:.1f}, Detected: {}".format(target,age,distance,kmag,full_detection_list[-1]))
    plt.scatter(comp_sep,comp_mass,label="Predicted Companion")

    ymin, ymax = axis.get_ylim()

    plt.xlabel("Separation (arcseconds)")
    plt.ylabel(r"Mass Limit ($M_{Jup}$)")
    plt.legend()
    plt.savefig(figure_directory+target+".png",dpi=200)
    print("Saving to {}".format(figure_directory+target+".png"))
    plt.close()

Target 0: eps Eri, 600.0 Myr, 3 pc, kmag=1.7


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


Saving to ./FarOut_Figures/Non-Detected/eps Eri.png
Target 1: GJ 674, 550.0 Myr, 5 pc, kmag=4.9
	Companion separation for GJ 674 is < 0.4 arcseconds, skipping
Target 2: GJ 514, 800.0 Myr, 8 pc, kmag=5.0
	Companion separation for GJ 514 is < 0.4 arcseconds, skipping
Target 3: AU Mic, 22.0 Myr, 10 pc, kmag=4.5
	Companion separation for AU Mic is < 0.4 arcseconds, skipping
Target 4: AU Mic, 22.0 Myr, 10 pc, kmag=4.5
	Companion separation for AU Mic is < 0.4 arcseconds, skipping
Target 5: L 98-59, 800.0 Myr, 11 pc, kmag=7.1
	Companion separation for L 98-59 is < 0.4 arcseconds, skipping
Target 6: L 98-59, 800.0 Myr, 11 pc, kmag=7.1
	Companion separation for L 98-59 is < 0.4 arcseconds, skipping
Target 7: L 98-59, 800.0 Myr, 11 pc, kmag=7.1
	Companion separation for L 98-59 is < 0.4 arcseconds, skipping
Target 8: L 98-59, 800.0 Myr, 11 pc, kmag=7.1
	Companion separation for L 98-59 is < 0.4 arcseconds, skipping
Target 9: HD 147513, 300.0 Myr, 13 pc, kmag=3.9
	Companion separation for HD 147

/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


	COMPANION DETECTED!
Saving to ./FarOut_Figures/Detected/GJ 504.png
Target 15: HN Peg, 300.0 Myr, 18 pc, kmag=4.6


...

/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]


	COMPANION DETECTED!
Saving to ./FarOut_Figures/Detected/CHXR 73.png
Target 120: CT Cha, 2.0 Myr, 191 pc, kmag=8.7


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]


	COMPANION DETECTED!
Saving to ./FarOut_Figures/Detected/CT Cha.png
Target 121: Kepler-63, 210.0 Myr, 194 pc, kmag=10.0
	Companion separation for Kepler-63 is < 0.4 arcseconds, skipping
Target 122: K2-105, 600.0 Myr, 198 pc, kmag=10.1
	Companion separation for K2-105 is < 0.4 arcseconds, skipping
Target 123: WASP-101, 900.0 Myr, 201 pc, kmag=9.1
	Companion separation for WASP-101 is < 0.4 arcseconds, skipping
Target 124: KELT-9, 300.0 Myr, 204 pc, kmag=7.5
	Companion separation for KELT-9 is < 0.4 arcseconds, skipping
Target 125: WASP-25, 20.0 Myr, 211 pc, kmag=10.2
	Companion separation for WASP-25 is < 0.4 arcseconds, skipping
Target 126: WASP-139, 500.0 Myr, 212 pc, kmag=10.5
	Companion separation for WASP-139 is < 0.4 arcseconds, skipping
Target 127: CoRoT-2, 600.0 Myr, 213 pc, kmag=10.3
	Companion separation for CoRoT-2 is < 0.4 arcseconds, skipping
Target 128: KELT-17, 650.0 Myr, 226 pc, kmag=8.6
	Companion separation for KELT-17 is < 0.4 arcseconds, skipping
Target 129: TrES-3, 

<ipython-input-92-3b463b6d64de>:20: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  print("Target {}: {}, {} Myr, {:.0f} pc, kmag={:.1f}".format(i,target,age,distance,kmag))
